<a href="https://colab.research.google.com/github/snikhil20/qwen2/blob/main/qwen2ebedddd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trial embedding-working

In [ ]:
# Step 1: Install the necessary packages
!pip install torch torchvision transformers
!pip install transformers accelerate
!pip install --upgrade transformers

import torch


In [ ]:
!pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-5e9szy0g
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-5e9szy0g
  Resolved https://github.com/huggingface/transformers to commit f38590dade57c1f8cf8a67e9409dae8935f8c478
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9706023 sha256=dc265e5c79f83bb6de20975d38159696ebcab4dce8b7397f9ee25f5a35639706
  Stored in directory: /tmp/pip-ephem-wheel-cache-dom_jwvd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
!pip install qwen-vl-utils

In [ ]:
import pandas as pd
import json
import requests
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import torch.nn as nn
import numpy as np

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
# Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# Load the processor for preprocessing
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()


# # Messages containing a video and text
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "video", "video": "https://path_to_your_video.mp4"},
#             {"type": "text", "text": "Get the embedding for this video."},
#         ],
#     }
# ]

# # Prepare inputs for inference
# text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# image_inputs, video_inputs = process_vision_info(messages)

# # If there are no images, set `images=[]`
# inputs = processor(
#     text=[text], images=[], videos=video_inputs, padding=True, return_tensors="pt"
# ).to("cuda")




# Messages containing both images, videos, and text (Example: video and image combined)
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "https://storage.googleapis.com/the-story-teller/originals/organizations/66a07f9ba8e8ba77305c22fe/platforms/649eccb00ef0f7c7c51a9daa/connectors/66a07fd7c98335bb5da262a8/images/66a0abe8c98335bb5da32626.jpg"},

            {"type": "text", "text": "Get the embedding for these media."},
        ],
    }
]

# Prepare inputs for inference
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt"
).to("cuda")

# Forward pass to obtain outputs
with torch.no_grad():
    outputs = model(**inputs)

# Explore different output attributes
# Check if the outputs include logits, embeddings, or any other relevant tensors
if hasattr(outputs, "logits"):
    embeddings = outputs.logits
elif hasattr(outputs, "encoder_last_hidden_state"):
    embeddings = outputs.encoder_last_hidden_state
elif hasattr(outputs, "pooler_output"):
    embeddings = outputs.pooler_output
else:
    raise ValueError("Unable to extract embeddings. Check model output attributes.")

# Detach the embeddings from the computational graph and move them to CPU if needed
embeddings = embeddings.detach().cpu()

print(embeddings)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.10 GiB. GPU 0 has a total capacity of 14.75 GiB of which 79.06 MiB is free. Process 17742 has 14.67 GiB memory in use. Of the allocated memory 13.24 GiB is allocated by PyTorch, and 1.30 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Clear the inputs from memory
del inputs
torch.cuda.empty_cache()
# Remove the batch dimension [1, 46, 152064] -> [46, 152064]
embeddings = embeddings.squeeze(0)

# Apply mean pooling across the 46 tokens to get a single representation
pooled_embeddings = torch.mean(embeddings, dim=0)  # Resulting shape [152064]

# Define a linear layer to project the pooled embedding to 768 dimensions
projection_layer = nn.Linear(152064, 768)

# Project the pooled embeddings to the desired shape [768]
final_embeddings = projection_layer(pooled_embeddings)

# Add the batch dimension back to get [1, 768]
final_embeddings = final_embeddings.unsqueeze(0)

print(final_embeddings.shape)  # torch.Size([1, 768])
print(final_embeddings)

torch.Size([1, 768])
tensor([[ 1.9769e+00, -3.4608e-01,  1.3764e+00,  1.0203e+00, -4.1091e-01,
         -1.7109e+00,  6.9818e-01,  7.3555e-01, -1.3626e+00,  1.7569e+00,
         -4.3670e-01, -3.3004e-01,  4.3814e-01, -2.6833e-01, -9.7841e-01,
         -5.5026e-01,  8.7861e-01,  1.4081e+00, -1.0601e+00, -3.6182e-01,
         -2.3331e+00, -1.8874e+00, -2.0664e+00, -3.5552e+00, -1.9114e+00,
         -3.3231e-01,  9.0156e-01, -1.6778e+00, -1.4735e+00, -9.2439e-01,
         -5.7699e-01,  3.4284e+00, -4.0535e-01, -1.3936e+00, -1.6071e-01,
          1.5454e+00, -7.8976e-01, -1.4565e+00, -5.8592e-01, -9.0327e-01,
          8.4759e-01, -1.4684e+00, -9.4106e-01, -9.5550e-01,  1.8803e-01,
         -1.5273e+00, -2.1243e+00,  9.9589e-01,  9.4121e-01, -3.8875e-01,
          1.0928e+00,  4.7286e-01,  8.3318e-01,  2.2484e-01,  1.8596e+00,
         -1.9002e-01,  6.5500e-01,  1.1591e+00,  8.6246e-01, -1.2388e+00,
          1.0704e-01,  2.5256e+00, -9.6177e-02, -4.1715e-01,  1.1454e+00,
         -7.1271e

In [ ]:
# Assuming 'final_embeddings' is your tensor with shape [1, 768]
# If the tensor is on the GPU, move it to the CPU first
if final_embeddings.is_cuda:
    final_embeddings = final_embeddings.cpu()

# Detach the tensor from the computation graph (if necessary)
final_embeddings = final_embeddings.detach()

# Convert the tensor to a numpy array
numpy_embeddings = final_embeddings.numpy()

print(numpy_embeddings.shape)  # Should print (1, 768)


(1, 768)


In [ ]:
print(numpy_embeddings)

[[-1.27013862e+00 -1.21240139e+00 -2.54351348e-01 -2.05937594e-01
   1.98247468e+00  2.11797261e+00 -1.69196486e+00 -3.83495283e+00
  -6.98050261e-01 -6.39389634e-01  1.63364142e-01  3.36719662e-01
   1.12391686e+00  2.18597817e+00  1.86186552e+00 -2.73572111e+00
   8.42638493e-01  1.65269482e+00 -5.94405711e-01  1.30378142e-01
   2.37781596e+00  1.44954062e+00 -2.73757529e+00 -1.21968329e-01
  -1.67323932e-01  2.35672995e-01  2.00517654e+00  6.00881457e-01
   1.12209308e+00 -3.61095905e-01 -3.90823752e-01  1.67745471e+00
   2.30535340e+00 -1.53596294e+00  2.15695649e-01 -5.09609270e+00
   9.86632109e-02 -1.58075735e-01  5.17899156e-01  2.86700845e-01
   1.30781174e+00  1.12382185e+00 -2.13414103e-01  2.42919922e+00
  -4.12560552e-01 -5.28938890e-01  6.58638835e-01 -4.17243361e-01
  -6.70196891e-01  8.40169251e-01 -1.98358297e+00 -1.41849768e+00
  -1.58303380e+00  9.91838455e-01 -5.50399661e-01 -3.80797505e-01
   3.22500086e+00 -9.61141139e-02  1.13469648e+00  1.07704151e+00
  -3.76134

#emdedding for the sheet

In [ ]:
import pandas as pd
import json
import requests
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import torch.nn as nn
import numpy as np
import gc

In [ ]:
# Load the Excel sheet
excel_file_path = "/content/creatives_data_done.xlsx"
creatives_df = pd.read_excel(excel_file_path)

# Load the JSON file with MediaAssetID to imageURL mapping
json_file_path = "/content/thestoryteller-prod.ad_images.json"
with open(json_file_path, 'r') as f:
    media_asset_mapping = json.load(f)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
# Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# Load the processor for preprocessing
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
def get_embedding_for_image_url(imageUrl):
    # Prepare the message structure for inference
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": imageUrl},
                {"type": "text", "text": "Get the embedding for this image."},
            ],
        }
    ]

    # Prepare inputs for inference
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Forward pass to obtain outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract embeddings from model outputs
    if hasattr(outputs, "logits"):
        embeddings = outputs.logits
    elif hasattr(outputs, "encoder_last_hidden_state"):
        embeddings = outputs.encoder_last_hidden_state
    elif hasattr(outputs, "pooler_output"):
        embeddings = outputs.pooler_output
    else:
        raise ValueError("Unable to extract embeddings. Check model output attributes.")

    # Detach the embeddings from the computational graph and move them to CPU
    embeddings = embeddings.detach().cpu()

    # Remove the batch dimension [1, 46, 152064] -> [46, 152064]
    embeddings = embeddings.squeeze(0)

    # Apply mean pooling across the 46 tokens to get a single representation
    pooled_embeddings = torch.mean(embeddings, dim=0)  # Resulting shape [152064]

    # Define a linear layer to project the pooled embedding to 768 dimensions
    projection_layer = nn.Linear(152064, 768)

    # Project the pooled embeddings to the desired shape [768]
    final_embeddings = projection_layer(pooled_embeddings)

    # Add the batch dimension back to get [1, 768]
    final_embeddings = final_embeddings.unsqueeze(0)

    # Convert the tensor to a numpy array
    if final_embeddings.is_cuda:
        final_embeddings = final_embeddings.cpu()

    final_embeddings = final_embeddings.detach().numpy()

    return final_embeddings

In [ ]:
# Initialize a list to store the embeddings
embeddings_list = []

# Iterate through each row in the Excel sheet
for idx, row in creatives_df.iterrows():
    # Clear GPU cache and collect garbage before processing each image
    gc.collect()
    torch.cuda.empty_cache()

    media_asset_id = row['mediaAssetId']  # Replace with the actual column name

    # Get the corresponding image URL from the JSON file
    imageUrl = None
    for item in media_asset_mapping: # Iterate through the list of dictionaries
        if str(item.get('adImageId')) == str(media_asset_id): # Check if the '_id' matches the media_asset_id
            imageUrl = item.get('imageUrl') # Get the 'adImage' value if there's a match
            break

    if imageUrl:
        # Get the embedding for the image URL
        try:
            embedding = get_embedding_for_image_url(imageUrl)
            embeddings_list.append(embedding)
        except Exception as e:
            print(f"Error processing {media_asset_id}: {e}")
            embeddings_list.append(None)
    else:
        print(f"No image URL found for {media_asset_id}")
        embeddings_list.append(None)

# Add the embeddings to the DataFrame
creatives_df['qwenCreatives_embedding'] = embeddings_list

# Save the updated DataFrame to a new Excel file
output_excel_file = "/content/updated_creative_data_done.xlsx"
creatives_df.to_excel(output_excel_file, index=False)

No image URL found for 1.31262E+14
No image URL found for 1.33726E+14
No image URL found for 1.48104E+14
No image URL found for 1.8519E+14
No image URL found for 1.92687E+14
No image URL found for 1004219044648659
No image URL found for 1031059927941825
No image URL found for 1077098530079689
No image URL found for 1091327755344104
No image URL found for 1096662267686199
No image URL found for 1211146752881571
No image URL found for 1216886908976606
No image URL found for 1252339839072579
No image URL found for 1320286915556569
No image URL found for 1323678541674327
No image URL found for 1328677788028339
No image URL found for 1357317441568789
No image URL found for 1365673364256792
No image URL found for 1370315127157611
No image URL found for 1420637011856181
No image URL found for 1429356967719662
No image URL found for 1510497946460082
No image URL found for 1522854338445473
No image URL found for 1532518434015837
No image URL found for 1539424133250066
No image URL found for 154

KeyboardInterrupt: 